In [ ]:
import tensorflow as tf 
import urllib.request
import matplotlib.pyplot as plt 
import os





In [ ]:
#下载泰坦尼数据集
data_url = "http:biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls"
data_file_path = "data/titanic3.xls"

if not os.path.isfile(data_file_path):
    result=urllib.urlretrieve(data_url,data_file_path)
    print('downloaded:',result)

In [ ]:
#读取数据
import numpy
import pandas as pd 
df_data = pd.read_excel(data_file_path)

In [ ]:
#筛选需要的字段
selected_cols = ['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
selected_df_data = df_data[selected_cols]

In [ ]:
selected_df_data.isnull().any()

In [ ]:
selected_df_data.isnull().sum()

In [ ]:
selected_df_data[selected_df_data.isnull().values=True]

In [ ]:
age_mean_value = selected_df_data['age'].mean()
selected_df_data['age'] = selected_df_data['age'].fillna(age_mean_value)

fare_mean_value = selected_df_data['fare'].mean()
selected_df_data['fare'] = selected_df_data['fare'].fillna(fare_mean_value)

selected_df_data['embarked'] = selected_df_data['embarked'].fillna('S')

selected_df_data['sex'] = selected_df_data['sex'].map({'female':0,'male':1}).astype(int)

selected_df_data['embarked'] = selected_df_data['embarked'].map({'C':0,'Q':1,'S':2}).astype(int)


In [ ]:
#删除name
selected_df_data = selected_df_data.drop(['name'],axis=1)

In [ ]:
selected_df_data[:3]

In [ ]:
#分离特征值和标签
ndarray_data = selected_df_data.values
features = ndarray_data[:,1:]
label = ndarray_data[:,0]

In [ ]:
#特征值标准处理化
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))
norm_features = minmax_scale.fit_transform(features)


In [ ]:
#定义数据预处理函数
def prepare_data(df_data):
    df=df_data.drop(['name'],axis=1)
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    df['sex'] = df['sex'].map({'female':0,'male':1}).astype(int)
    df['embarked'] = df['embarked'].fillna('S')
    df['embarked'] = df['embarked'].map({'C':0,'Q':1,'S':2}).astype(int)

    ndarray_data = df.values

    features = ndarray_data[:,1:]
    label = ndarray_data[:,0]

    minmax_scale =preprocessing.MinMaxScaler(feature_range=(0,1))
    norm_features=minmax_scale.fit_transform(features)

In [ ]:
#数据打乱顺序，划分训练集
shuffled_df_data = selected_df_data.sample(frac=1)

x_data,y_data = prepare_data(shuffled_df_data)

train_size = int(len(x_data)*0.8)

x_train = x_data[:train_size]
y_train = y_data[:train_size]

x_test = x_data[train_size:]
y_test  =y_data[train_size:]

In [2]:
import tensorflow as tf
#建立keras序列模型
model = tf.keras.models.Sequential()

In [ ]:
#加入第一层
model.add(tf.keras.layers.Dense(
                                units=64,
                                input_dim=7,
                                use_bias=True,
                                kernel_initializer='uniform',
                                bias_initializer='zero'
                                activation='relu'
))
model.add(tf.keras.layers.Dense(units=32,
                                activation='sigmoid'))
model.add(tf.karas.layers.Dense(units=1,
                                activation='sigmoid'))

In [ ]:
#模型设置
model.compile(optimizer=tf.keras.optimizers.Adam(0.003),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#模型训练
train_history = model.fit(x=x_train,
                          y=y_ttain,
                          validation_split=0.2,
                          epochs=100,
                          batch_size=40,
                          verbose=2)

In [ ]:
#训练过程可视化
import matplotlib as plt

def visu_train_history(train_history,train_metric,validation_metric):
    plt.plot(train_history.history[train_metric])
    plt.plot(train_history.history[validation_metric])
    plt.title('Train History')
    plt.ylabel('train_metric')
    plt.xlabel('epoch')
    plt.legend(['train','validation'],loc='upper left')
    plt.show()

In [ ]:
visu_train_history(train_history,'acc',val_acc)

In [ ]:
visu_train_history(train_history,'loss',val_loss)

In [ ]:
#模型评估
evaluate_result = model.evaluate(x=x_test,y_y_test)

In [ ]:
evaluate_result

In [ ]:
#评估结果返回标签
model.metrics_names

In [ ]:
Jack_info = [0,'Jack','male',23,1,0,5.000,'S']
Rose_info = [1,'Rose',1,'female',20,1,0,100.000,'S']
new_passenger_pd=pd.DataFrame([Jack_info,Rose_info],columns=selected_cols)

In [ ]:
all_passenger_pd = selected_df_data.append(new_passenger_pd)

In [ ]:
all+passenger_pd[-3:]

In [ ]:
x_features,y_label = prepare_data(all-passenger_pd)
surv_probability = model.predict(x_features)
surv_probability[:5]

In [ ]:
all_passenger_pd.insert(len(all_passenger_pd.columns),'surv_probability',surv_probability)
all_passenger_pd[-5:]